# Datasetの実装


## データセットのダウンロード

In [46]:
import os
import urllib.request
import zipfile
import tarfile
import os.path as osp
from os.path import expanduser
home = expanduser("~")
import numpy as np

In [47]:
data_dir = os.path.join(home, "data/")        
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [48]:
weights_dir = "./weights"
if not os.path.exists(weights_dir):
    os.mkdir(weights_dir)
    

In [49]:
url = "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar"
target_path = os.path.join(data_dir, "VOCtrainval_11-May-2012.tar")
#print(os.path.exists(target_path))

if not os.path.exists(target_path):
    urllib.request.urlretrieve(url, target_path)
    
    tar = tarfile.TarFile(target_path)  #tarfile読み込み
    tar.extractall(data_dir)            #tarを解凍
    tar.close                           #tarをクローズ
    

## データセット作成

・訓練時のDataAugumentationの際に、アノテーションデータも変換する必要があるのに注意

画像とアノテーションデータへのファイルパスリストを作成

In [50]:
def make_datapath_list(rootpath):
    
    
    imgpath_template = osp.join(rootpath, "JPEGImages", "%s.jpg")
    annopath_template = osp.join(rootpath, "Annotations", "%s.xml")
    
    train_id_names = osp.join(rootpath, "ImageSets/Main/train.txt")
    val_id_names = osp.join(rootpath, "ImageSets/Main/val.txt")
    
    train_img_list=list()
    train_anno_list = list()
    
    for line in open(train_id_names):
        file_id = line.strip()  #空白スペースと改行を除去
        img_path=(imgpath_template % file_id)                   #「%s」の部分にfile_idを代入
        anno_path=(annopath_template % file_id)
        train_img_list.append(img_path)
        train_anno_list.append(anno_path)
        
    val_img_list = list()
    val_anno_list = list()
    
    for line in open(val_id_names):
        file_id = line.strip()
        img_path = (imgpath_template % file_id)
        anno_path = (annopath_template % file_id)
        val_img_list.append(img_path)
        val_anno_list.append(anno_path)
        
    return train_img_list, train_anno_list, val_img_list, val_anno_list
        
        
        
    

In [51]:
rootpath = osp.join(data_dir + "/VOCdevkit/VOC2012")
train_img_list, train_anno_list, val_img_list, val_anno_list = make_datapath_list(rootpath)

#動作確認
print(train_img_list[0])

/home/yoshiki/data//VOCdevkit/VOC2012/JPEGImages/2008_000008.jpg


xml形式のアノテーションデータをリストに変換  
・クラス名を文字列から数値へと置き換える  
・アノテーションデータ（バウンディングボックスの座標）を画像サイズで規格化する

In [52]:
# XMLをファイルやテキストから読み込んだり、加工したり、保存したりするためのライブラリ
import xml.etree.ElementTree as ET

In [53]:
#XML形式のアノテーションを，リスト形式に変換する
class Anno_xml2list(object):
    
    def __init__(self, classes):
        self.classes = classes
        
    def __call__(self, xml_path, width, height):
        
        ret = []                #[[xmin, ymin, xmax, ymax, label_index], ・・・]
        
        xml = ET.parse(xml_path).getroot()   #xmlファイルを読み込む
        
        for obj in xml.iter("object"):      #画像内のobjectの数だけループする
            
            #アノテーションで検知がdifficultに設定されているものは除外
            difficult = int(obj.find("difficult").text)
            if difficult:
                continue
            
            bndbox = []                                   #1つの物体に対するアノテーションを格納するリスト
            
            name = obj.find("name").text.lower().strip()  #物体名(str.lower():すべての文字を小文字に変換. / str.strip():両端の指定の文字を削除)
            bbox = obj.find("bndbox")                     #バウンディングボックスの情報
            
            pts = ["xmin", "ymin", "xmax", "ymax"]
            
            for pt in (pts):
                cur_pixel = int(bbox.find(pt).text)-1    #vocデータは原点が(1,1)なので、1を引いて(0,0)にする
                
                if pt == "xmin" or pt == "xmax":
                    cur_pixel /= width                  #x方向は幅で割る
                else:
                    cur_pixel /= height

                bndbox.append(cur_pixel)

            label_idx = self.classes.index(name)
            bndbox.append(label_idx)

            ret += [bndbox]
            
        return np.array(ret)
            
            

            
        
        
        

In [60]:
import cv2

voc_classes=["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", 
             "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

transform_anno=Anno_xml2list(voc_classes)

test_img=cv2.imread(val_img_list[1])
print(transform_anno(val_anno_list[1], test_img.shape[1], test_img.shape[0]))


#img=cv2.imread("path")   img.shape=height, width, channels

#xmlファイルの中身の確認
"""
XML=ET.parse(xml_path).getroot()
for child in XML:
    print(child.tag, child.attrib)
"""


[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]


'\nXML=ET.parse(xml_path).getroot()\nfor child in XML:\n    print(child.tag, child.attrib)\n'

### 画像とアノテーションの前処理を行うクラスDataTransformを作成
訓練時はデータオーグメンテーション
- 色調を変換し，画像の大きさを変更してからランダムに切り出す．さらに画像の大きさをリサイズし，色情報の平均値を引き算


推論時は画像の大きさを変換し，色情報の平均値を引き算するだけ

In [ ]:
from utils.data_augumentation import Compose, ConvertFromInts, ToAbsoluteCoords, PhotometricDistort, Expand, RandomSampleCrop, RandomMirror, ToPercentCoords, Resize, SubtractMeans

class DataTransform():
    def __init__(self, input_size, color_mean):
        self.data_transform = {
            
        }
